

**Reference**:
https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing#scrollTo=pCqnaKmlO1U9


**This script contains**

1. finetuning of llama3.1 model: save to gdrive by default
2. inference: abtain w2r_result.json file, save to gdrive by default

**Note**

Only extract waste and transformed_resource
and add transforming_process as an empty list

**Before start**

- runtime change to GPU T4


## Prepare packages and models

In [ ]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.8/161.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install bitsandbytes
!pip install xformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install -U "huggingface_hub[cli]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

# this is used to download model from huggingface

In [ ]:
# this takes around 9mins
# model size > 16GB

# !huggingface-cli download meta-llama/Meta-Llama-3.1-8B-Instruct --repo-type model --local-dir "/content/drive/MyDrive/Colab Notebooks/W2R/Meta-Llama-3.1-8B-Instruct"

.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 8.89MB/s]
Download complete. Moving file to /content/drive/MyDrive/Colab Notebooks/W2R/Meta-Llama-3.1-8B-Instruct/.gitattributes
LICENSE: 100% 7.63k/7.63k [00:00<00:00, 40.1MB/s]
Download complete. Moving file to /content/drive/MyDrive/Colab Notebooks/W2R/Meta-Llama-3.1-8B-Instruct/LICENSE
README.md: 100% 44.0k/44.0k [00:00<00:00, 35.3MB/s]
Download complete. Moving file to /content/drive/MyDrive/Colab Notebooks/W2R/Meta-Llama-3.1-8B-Instruct/README.md
USE_POLICY.md: 100% 4.69k/4.69k [00:00<00:00, 39.1MB/s]
Download complete. Moving file to /content/drive/MyDrive/Colab Notebooks/W2R/Meta-Llama-3.1-8B-Instruct/USE_POLICY.md
config.json: 100% 855/855 [00:00<00:00, 6.63MB/s]
Download complete. Moving file to /content/drive/MyDrive/Colab Notebooks/W2R/Meta-Llama-3.1-8B-Instruct/config.json
generation_config.json: 100% 184/184 [00:00<00:00, 1.65MB/s]
Download complete. Moving file to /content/drive/MyDrive/Colab Notebooks/W2R/Meta-Llama-3.1-8B-

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

orig_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/Colab Notebooks/W2R//Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/content/drive/MyDrive/Colab Notebooks/W2R//Meta-Llama-3.1-8B-Instruct does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


## Inference - test initial performance

In [ ]:
from datasets import Dataset
import json
import os

In [ ]:
FastLanguageModel.for_inference(orig_model) # Enable native 2x faster inference

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,),

Requirments:
- Perform extraction by yourself and only assign the results to the corresponding keys.
- Extract only names or phrases that describe wastes and transformed resources.
- If no wastes or resources are found, keep the corresponding list empty.
- Do not include any explanations or comments; do not import any libraries; output only the additional code needed to perform the extraction.



In [ ]:
def make_instruction_from_abstract():
    instruction_template = """Complete the code by extracting the waste-to-resource information from the given paragraph.
You should extract related waste and transformed resources as lists and assign them under keys of "waste" and "tranformed_resource", respectively.
Do not import libraries. Do not use abbreviations. Do not explain yourself. Directly output the newly added code. """
    return instruction_template

def make_coded_abstract(abstract):
  coded_abstract = """
```python
def extract():
  text = "{abstract}"
  w2r = {{"waste": [], "transformed_resource": []}}
  # now read the text, assign wastes and resources lists to w2r keys
  # directly output the newly added code
```""".format(abstract=abstract)
  return coded_abstract

def make_formatted_prompt(abstract)-> str:
  instruction = make_instruction_from_abstract()
  input_x = make_coded_abstract(abstract)

  llama31_prompt="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{}<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

  formatted_prompt = llama31_prompt.format(instruction, input_x)
  return formatted_prompt


test one example

In [ ]:
from transformers import TextIteratorStreamer

abstract = "The pyrolysis and co-pyrolysis characteristics of large abundant bio-wastes, namely date pits (DP), peanut shells (PS), coffee grounds (CG), and tea waste (TW) were investigated in detail. The TG/DTG and pyrolysis procedures were used to examine the thermal pyrolysis behavior of these agro-wastes and their blends (50%CG/50%DP, 50%CG/50%PS, 50%DP/50%PS, 50%TW/50% DP,50%TW/50%PS and 50%CG/50% TW by weight). The characterization of these feedstock has shown their suitability for energy and material valorization. Experiments were carried out in a batch pyrolysis at a medium heating rate of 10 °C/min. The goal was to identify the best combination aiming to produce either char or hydrogen-rich gas, which can contribute to the development of the circular bio-economy. It was demonstrated that pyrolysis favored the liquid product, fluctuating from 34.31% (TW) to 50.92% (DP), while the co-pyrolysis greatly increased the gaseous product, which ranged between 42.02% (TW-DP) and 55.17% (PS-TW). The blending of CG and PS resulted in heightened reactivity, leading to an enhanced generation of H2 (34.44%). The optimal mixture was found to be CG-TW, showcasing superior performance in terms of gas quality (38.39% H2) and yield (53.74%). This outcome underscores the potential of CG and TW as a synergistic blend for efficient hydrogen-rich gas production. The FTIR findings revealed that the recovered biochars have the potential to serve as solid fuels, biofertilizers, or carbon materials. Additionally, they could be used as eco-friendly precursors for chemical and related industries. By analyzing various waste mixtures and their respective pyrolysis properties, this research aims to contribute to the development of sustainable waste management practices as well as efficient energy and material production methods. © 2024 The Institution of Chemical Engineers"
inputs_string = make_formatted_prompt(abstract)

inputs = tokenizer([inputs_string], return_tensors="pt").to("cuda")

# Initialize the TextIteratorStreamer
text_iter_streamer = TextIteratorStreamer(tokenizer)

# Generate the content using the model and the streamer
_ = orig_model.generate(**inputs, streamer=text_iter_streamer, max_new_tokens=1024)

# Collect the generated tokens from the streamer and join them into a single string
generated_text = "".join([token for token in text_iter_streamer])

# Remove the original input part from the generated text, if needed
original_input_text = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)
generated_text = generated_text[len(original_input_text):].strip()

# Print or use the generated content
print("Result: ")
print(generated_text)

Result: 
w read the text, assign wastes and resources lists to w2r keys
  # directly output the newly added code
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>

```python
def extract():
  text = "The pyrolysis and co-pyrolysis characteristics of large abundant bio-wastes, namely date pits (DP), peanut shells (PS), coffee grounds (CG), and tea waste (TW) were investigated in detail. The TG/DTG and pyrolysis procedures were used to examine the thermal pyrolysis behavior of these agro-wastes and their blends (50%CG/50%DP, 50%CG/50%PS, 50%DP/50%PS, 50%TW/50% DP,50%TW/50%PS and 50%CG/50% TW by weight). The characterization of these feedstock has shown their suitability for energy and material valorization. Experiments were carried out in a batch pyrolysis at a medium heating rate of 10 °C/min. The goal was to identify the best combination aiming to produce either char or hydrogen-rich gas, which can contribute to the development of the circular bio-economy. It was demonstrated t

run on test abstracts and save result

In [ ]:
import pandas as pd
import logging
import json
import time
import ast
import re

In [ ]:
def extract(abstract):
  inputs_string = make_formatted_prompt(abstract)
  inputs = tokenizer([inputs_string], return_tensors="pt").to("cuda")

  # Initialize the TextIteratorStreamer
  text_iter_streamer = TextIteratorStreamer(tokenizer)

  # generate
  _ = orig_model.generate(**inputs, streamer=text_iter_streamer, max_new_tokens=1024)
  generated_text = "".join([token for token in text_iter_streamer])

  # Remove the original input part from the generated text, if needed
  original_input_text = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)
  generated_text = generated_text[len(original_input_text):].strip()

  return generated_text


def extract_postprocessing(result):
  w2r = {"waste": [], "transforming_process": [], "transformed_resource": []}
  try:
    # Regular expression pattern to match w2r assignments
    pattern = r'w2r\["(?P<key>\w+)"\]\s*=\s*(?P<value>\[.*?\])'
    # pattern = r'(?:w2r\["(?P<key>\w+)"\]|(?P<key>\w+))\s*=\s*(?P<value>\[.*?\])'
    # Find all matches in the code string
    matches = re.finditer(pattern, result, re.DOTALL)
    for match in matches:
      key = match.group('key')
      value_str = match.group('value')
      try:
        # Safely evaluate the list using ast.literal_eval
        value = ast.literal_eval(value_str)
        if isinstance(value, list):
          w2r[key] = value
        else:
          print(f"Warning: The value for '{key}' is not a list.")
      except Exception as e:
        print(f"Error evaluating the list for '{key}': {e}")
  except Exception as e:
    print("An error occurred while parsing the code string:", e)
  return w2r


def check_make_dir(directory, exist_ok=False):
  if not os.path.exists(directory):
    os.makedirs(directory)
    return directory
  else:
    if not exist_ok:
      i = 1
      # increment until finding an available name
      new_dir = f"{directory}_{i}"
      while os.path.exists(new_dir):
        i += 1
        new_dir = f"{directory}_{i}"
      os.makedirs(new_dir)
      return new_dir
    return directory

In [ ]:
def process_abstracts(args):

  # num, model, prompt, save_dir, shot_length, shot_k

  result_json_file = os.path.join(args.save_dir, "w2r_results.json")
  result_invalid_file = os.path.join(args.save_dir, "w2r_invalid.txt")
  result_invalid_doi_file = os.path.join(args.save_dir, "w2r_invalid_doi.txt")

  df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/W2R/scopus_waste2resource.csv")
  abstract_list = df["Abstract"].tolist()
  doi_list = df["DOI"].tolist()

  if args.start_index:
    abstract_list = abstract_list[args.start_index:]
    doi_list = doi_list[args.start_index:]

  if args.num > 0:
    abstract_list = abstract_list[:args.num]
    doi_list = doi_list[:args.num]

  result_json_list = []
  result_invalid_str = ""
  result_invalid_doi_list = []

  total_time = 0

  for i, abstract in enumerate(abstract_list):
    start_time = time.time()

    # get llm output and process
    coded_abstract = abstract   # no need to wrap
    result_str = extract(coded_abstract)
    result_json = extract_postprocessing(result_str)

    # add the referece and append
    result_json["reference"] = doi_list[i]
    result_json_list.append(result_json)

    # check if the extracted information is valid
    if len(result_json["waste"]) == 0 and len(result_json["transforming_process"]) == 0 and len(result_json["transformed_resource"]) == 0:
      result_invalid_str += result_str + "\n\n----------\n\n"
      result_invalid_doi_list.append(doi_list[i])

    logging.info("----------------------------------")
    logging.info("Processing abstract {}".format(i))
    logging.info("Coded abstract: \n{}".format(coded_abstract))
    logging.info("LLM response: \n{}".format(result_str))
    logging.info("Extracted w2r: \n{}".format(result_json))

    end_time = time.time()
    total_time += end_time - start_time
    logging.info("Time taken: {:.2f} seconds".format(end_time - start_time))
    print("Processing abstract {}, time taken: {:.2f} seconds".format(i, end_time - start_time))
    # end for loop

    # save results each 10 abstracts
    if i % 5 == 0:
      with open(result_json_file, 'w') as file:
        json.dump(result_json_list, file, indent=4)

  # save the final result
  with open(result_json_file, 'w') as file:
    json.dump(result_json_list, file, indent=4)

  try:
    with open(result_invalid_file, 'w') as file:
      file.write(result_invalid_str)
  except:
    with open(result_invalid_file, 'w', encoding='utf-8') as file:
      file.write(result_invalid_str)

  with open(result_invalid_doi_file, 'w') as file:
    for invalid_doi in result_invalid_doi_list:   # if empty, doi = nan, then need to convert to string
      file.write(str(invalid_doi) + '\n')

  logging.info("-------------"*5)
  logging.info("Successfully extracted: {} / {}".format(len(result_json_list), len(abstract_list)))
  logging.info("Total time: {}, averaged time: {} seconds".format(total_time, total_time/len(result_json_list)))


In [ ]:
class Args:
  def __init__(self):
    self.num = 50
    self.start_index = None
    self.save_dir = "/content/drive/MyDrive/Colab Notebooks/W2R/extraction_finetune_unsloth/result_codestyle_basic"
    self.save_dir_rewrite = True
    self.repeat = 5

# Create an instance of Args
args = Args()
base_folder = args.save_dir

# Run the process_abstracts function
for i in range(args.repeat):
  print("Running experiment {}/{}".format(i+1, args.repeat))
  logging.info("Running experiment {}/{}".format(i+1, args.repeat))


  # create subfolder in the save_dir: run_{i}
  save_dir = os.path.join(base_folder, "run_{}".format(i))
  args.save_dir = check_make_dir(save_dir, exist_ok=args.save_dir_rewrite)

  # set logging file
  logger = logging.getLogger()
  if logger.hasHandlers():
    logger.handlers.clear()
  log_file = os.path.join(args.save_dir, "extraction_log.log")
  logging.basicConfig(filename=log_file, filemode='w', level=logging.INFO, format='%(message)s')

  # Log each argument and its value
  for arg, value in vars(args).items():
    logging.info(f'Argument {arg}: {value}')
    print(f'{arg}: {value}')
  logging.info("-------------"*5)

  process_abstracts(args)

## Finetune model

In [ ]:
model = FastLanguageModel.get_peft_model(
    model = orig_model,
    r = 8,   # higher r allow model to learn more parameters
    target_modules = ["q_proj", "k_proj", "v_proj",   # query, key, value projections in the attention layers
                      "o_proj",      # output projection of the attention
                      "gate_proj", "up_proj", "down_proj"],  # related to gating mechanisms within the model
    lora_alpha = 32,   # control how much the learned low-rank matrics affect the original weights
    lora_dropout = 0.2,
    bias = "none",    # none / all / lora_only
    use_gradient_checkpointing = "unsloth",  # unsloth / True. unsloth reduce VRAM by 30%
    random_state = 55,
    use_rslora = False,    # rank stablised LoRA
    loftq_config = None,   # LoftQ: low-rank and fine-tuned quantizatioin
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.2.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.11.5 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


### Data preparation

In [ ]:
from datasets import Dataset
import json
import os

In [ ]:

def make_instruction_from_abstract():
    instruction_template = """Complete the code by extracting the waste-to-resource information from the given paragraph.
You should extract related waste and transformed resources as lists and assign them under keys of "waste" and "tranformed_resource", respectively.
Directly output the complete code. Do not import libraries. Do not use abbreviations. Do not explain yourself."""
    return instruction_template

def make_coded_abstract(abstract):
  coded_abstract = """
```python
def extract():
  text = "{abstract}"
  w2r = {{"waste": [], "transformed_resource": []}}
  # now read the text, assign wastes and resources lists to w2r keys
```""".format(abstract=abstract)
  return coded_abstract

def make_coded_result(result):
  coded_result = """
```python
w2r["waste"] = {}
w2r["transformed_resource"] = {}
```
""".format(json.dumps(result["waste"]), json.dumps(result["transformed_resource"]))
  return coded_result

def make_formatted_prompt(abstract, result)-> str:
  instruction = make_instruction_from_abstract()
  input_x = make_coded_abstract(abstract)
  result_str = make_coded_result(result)

  llama31_prompt="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{}<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{}<|eot_id|>"""

  formatted_prompt = llama31_prompt.format(instruction, input_x, result_str)
  return formatted_prompt

In [ ]:

EOS_TOKEN = tokenizer.eos_token  # must add this, otherwise the generation will not stop

def prepare_dataset(annotation_file):
  with open(annotation_file, "r", encoding="utf-8") as f:
    data = json.load(f)

  # format
  formatted_data = []
  for entry in data:
    abstract = entry["abstract"]
    result = {"waste": entry["waste"], "transformed_resource": entry["transformed_resource"]}

    text = make_formatted_prompt(abstract, result) + EOS_TOKEN  # MAYBE DON'T NEED EOS_TOKEN
    formatted_data.append({"text":text})

  dataset = Dataset.from_list(formatted_data)
  return dataset

In [ ]:
# annotation_file = "/content/finetune_dataset.json"
annotation_file = "/content/drive/MyDrive/Colab Notebooks/W2R/extraction_finetune_dataset.json"
with open(annotation_file, "r", encoding="utf-8") as f:
  data = json.load(f)

train_data = prepare_dataset(annotation_file)

In [ ]:
print(train_data[2])

{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nComplete the code by extracting the waste-to-resource information from the given paragraph.\nYou should extract related waste and transformed resources as lists and assign them under keys of "waste" and "tranformed_resource", respectively.\nDirectly output the complete code. Do not import libraries. Do not use abbreviations. Do not explain yourself.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n\n```python\ndef extract():\n  text = "Extracting functional materials from industrial waste residues to absorb organic dyes can maximize waste reuse and minimize water pollution. However, the extraordinarily low purification efficiency still limits the practical application of this strategy. Herein, the lamellar NiOOH is in-situ anchored on the industrial waste red mud surface (ARM/NiOOH) as an adsorbent to purify organic dyes in wastewater. ARM/NiOOH adsorbent with high specific surface area and porosity provides c

In [ ]:
print(len(train_data))

100


### Train the model and save to GDrive

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported   # check if hardware supports bfloat16 precision
import time

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    dataset_text_field = "text",     # formatted_data.append({"text":text})
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,  # num of processes for dataset preprocessing, which speeds up data preparation
    packing = False,  # if True, combine shorter sequences to fill each batch, can increase training speed
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 10, # set this for 1 full training run,
        # max_steps = 60,     # otherwise, set maximum number of training steps
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 55,
        output_dir = "output",
        report_to = "none",  # can export to services like Weights & Biases or TensorBoard
    )
)

Map (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


In [ ]:
import time

In [ ]:
model_name = "finetuned_model_codestyle4"
base_dir = "/content/drive/MyDrive/Colab Notebooks/W2R/extraction_finetune_unsloth"   # save to Gdrive

start_time = time.time()

trainer_stats = trainer.train()

end_time = time.time()
print(f"Training time: {(end_time - start_time)/60} minutes")


model.save_pretrained(os.path.join(base_dir, model_name))
tokenizer.save_pretrained(os.path.join(base_dir, model_name))
print("Finetuned model saved at: ", os.path.join(base_dir, model_name))

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 120
 "-____-"     Number of trainable parameters = 20,971,520


Step,Training Loss
1,2.708500
2,2.847700
3,2.624500
4,2.477500
5,2.440400
6,2.323200
7,2.013500
8,2.091300
9,1.825400
10,1.853900


Training time: 25.03040300210317 minutes
Finetuned model saved at:  /content/drive/MyDrive/Colab Notebooks/W2R/extraction_finetune_unsloth/finetuned_model_codestyle4


In [ ]:
# Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1494.8971 seconds used for training.
24.91 minutes used for training.
Peak reserved memory = 7.982 GB.
Peak reserved memory for training = 1.998 GB.
Peak reserved memory % of max memory = 54.123 %.
Peak reserved memory for training % of max memory = 13.548 %.


## Inference

**load fine-tuned model**

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [ ]:
from unsloth import FastLanguageModel
import os

model_name = "finetuned_model_codestyle3"
base_dir = "/content/drive/MyDrive/Colab Notebooks/W2R/extraction_finetune_unsloth"   # save to Gdrive

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = os.path.join(base_dir, model_name), # MODEL NAME here
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Unsloth 2024.11.5 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
          

In [ ]:
def make_instruction_from_abstract():
    instruction_template = """Complete the code by extracting the waste-to-resource information from the given paragraph.
You should extract related waste and transformed resources as lists and assign them under keys of "waste" and "tranformed_resource", respectively.
Do not import libraries. Do not use abbreviations. Do not explain yourself. Directly output the newly added code. """
    return instruction_template

def make_coded_abstract(abstract):
  coded_abstract = """
```python
def extract():
  text = "{abstract}"
  w2r = {{"waste": [], "transformed_resource": []}}
  # now read the text, assign wastes and resources lists to w2r keys
  # directly output the newly added code
```""".format(abstract=abstract)
  return coded_abstract

def make_formatted_prompt(abstract)-> str:
  instruction = make_instruction_from_abstract()
  input_x = make_coded_abstract(abstract)

  llama31_prompt="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{}<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

  formatted_prompt = llama31_prompt.format(instruction, input_x)
  return formatted_prompt


**Test one example**

In [ ]:
from transformers import TextIteratorStreamer

abstract = "The valorization of tannins from forest by-products is a major opportunity for the sustainable development of the forestry industry. Herein we report the laccase-mediated polymerization of tannins obtained from an aqueous pine bark extract. The reaction was conducted under an oxygen atmosphere at 50°C for 60 min and followed by UV–Vis, fluorescence intensity, phenol content, and viscosity measurements. An insoluble polymer was obtained after 45 min, which was attributed to coupling reactions triggered by phenoxyl radicals within the tannins mixture. The polymer showed a marked increase in thermal stability compared with the precursor pine bark extract, with a total mass loss of less than 12% at temperatures up to 500°C, suggesting potential practical applications in flame retardant materials. Additionally, the polymer showed an 85% DPPH activity and a 60% attenuation of singlet oxygen release upon light irradiation, which supports antioxidant and photoprotective properties for the prepared material. To the best of our knowledge, this is the first report to describe the formation of an insoluble polymer by direct enzymatic reaction of an aqueous tannin extract. In this case, soluble tannins are the sole reactants used to produce a promising versatile material for forest waste valorization. © 2024 Wiley Periodicals LLC."
inputs_string = make_formatted_prompt(abstract)

inputs = tokenizer([inputs_string], return_tensors="pt").to("cuda")

# Initialize the TextIteratorStreamer
text_iter_streamer = TextIteratorStreamer(tokenizer)

# Generate the content using the model and the streamer
_ = model.generate(**inputs, streamer=text_iter_streamer, max_new_tokens=256)

# Collect the generated tokens from the streamer and join them into a single string
generated_text = "".join([token for token in text_iter_streamer])

# Remove the original input part from the generated text, if needed
original_input_text = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)
generated_text = generated_text[len(original_input_text):].strip()

# Print or use the generated content
print("Result: ")
print(generated_text)

Result: 
w read the text, assign wastes and resources lists to w2r keys
  # directly output the newly added code
```<|eot_id|><|start_header_id|>assistant<|end_header_id|>


```python
w2r["waste"] = ["forest by-products"]
w2r["transformed_resource"] = ["polymer"]
```
<|eot_id|>


**Inference on 50 abstracts**

In [ ]:
import pandas as pd
import logging
import json
import time
import ast
import re

define some functions

In [ ]:
def extract(abstract, args):
  inputs_string = make_formatted_prompt(abstract)
  inputs = tokenizer([inputs_string], return_tensors="pt").to("cuda")

  # Initialize the TextIteratorStreamer
  text_iter_streamer = TextIteratorStreamer(tokenizer)

  # generate
  _ = model.generate(**inputs, streamer=text_iter_streamer, max_new_tokens=512)
  generated_text = "".join([token for token in text_iter_streamer])

  # Remove the original input part from the generated text, if needed
  original_input_text = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)
  generated_text = generated_text[len(original_input_text):].strip()

  return generated_text

def extract_postprocessing(result):
  w2r = {"waste": [], "transforming_process": [], "transformed_resource": []}
  try:
    # Regular expression pattern to match w2r assignments
    pattern = r'w2r\["(?P<key>\w+)"\]\s*=\s*(?P<value>\[.*?\])'
    # Find all matches in the code string
    matches = re.finditer(pattern, result, re.DOTALL)
    for match in matches:
      key = match.group('key')
      value_str = match.group('value')
      try:
        # Safely evaluate the list using ast.literal_eval
        value = ast.literal_eval(value_str)
        if isinstance(value, list):
          w2r[key] = value
        else:
          print(f"Warning: The value for '{key}' is not a list.")
      except Exception as e:
        print(f"Error evaluating the list for '{key}': {e}")
  except Exception as e:
    print("An error occurred while parsing the code string:", e)
  return w2r

def check_make_dir(directory, exist_ok=False):
  if not os.path.exists(directory):
    os.makedirs(directory)
    return directory
  else:
    if not exist_ok:
      i = 1
      # increment until finding an available name
      new_dir = f"{directory}_{i}"
      while os.path.exists(new_dir):
        i += 1
        new_dir = f"{directory}_{i}"
      os.makedirs(new_dir)
      return new_dir
    return directory

In [ ]:
def process_abstracts(args):

  # num, model, prompt, save_dir, shot_length, shot_k

  result_json_file = os.path.join(args.save_dir, "w2r_results.json")
  result_invalid_file = os.path.join(args.save_dir, "w2r_invalid.txt")
  result_invalid_doi_file = os.path.join(args.save_dir, "w2r_invalid_doi.txt")

  df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/W2R/scopus_waste2resource.csv")
  abstract_list = df["Abstract"].tolist()
  doi_list = df["DOI"].tolist()

  if args.start_index:
    abstract_list = abstract_list[args.start_index:]
    doi_list = doi_list[args.start_index:]

  if args.num > 0:
    abstract_list = abstract_list[:args.num]
    doi_list = doi_list[:args.num]

  result_json_list = []
  result_invalid_str = ""
  result_invalid_doi_list = []

  total_time = 0

  for i, abstract in enumerate(abstract_list):
    start_time = time.time()

    # get llm output and process
    coded_abstract = abstract   # no need to wrap
    result_str = extract(coded_abstract, args)
    result_json = extract_postprocessing(result_str)

    # add the referece and append
    result_json["reference"] = doi_list[i]
    result_json_list.append(result_json)

    # check if the extracted information is valid
    if len(result_json["waste"]) == 0 and len(result_json["transforming_process"]) == 0 and len(result_json["transformed_resource"]) == 0:
      result_invalid_str += result_str + "\n\n----------\n\n"
      result_invalid_doi_list.append(doi_list[i])

    logging.info("----------------------------------")
    logging.info("Processing abstract {}".format(i))
    logging.info("Coded abstract: \n{}".format(coded_abstract))
    logging.info("LLM response: \n{}".format(result_str))
    logging.info("Extracted w2r: \n{}".format(result_json))

    end_time = time.time()
    total_time += end_time - start_time
    logging.info("Time taken: {:.2f} seconds".format(end_time - start_time))
    print("Processing abstract {}, time taken: {:.2f} seconds".format(i, end_time - start_time))
    # end for loop

    # save results each 10 abstracts
    if i % 10 == 0:
      with open(result_json_file, 'w') as file:
        json.dump(result_json_list, file, indent=4)

  # save the final result
  with open(result_json_file, 'w') as file:
    json.dump(result_json_list, file, indent=4)

  try:
    with open(result_invalid_file, 'w') as file:
      file.write(result_invalid_str)
  except:
    with open(result_invalid_file, 'w', encoding='utf-8') as file:
      file.write(result_invalid_str)

  with open(result_invalid_doi_file, 'w') as file:
    for invalid_doi in result_invalid_doi_list:   # if empty, doi = nan, then need to convert to string
      file.write(str(invalid_doi) + '\n')

  logging.info("-------------"*5)
  logging.info("Successfully extracted: {} / {}".format(len(result_json_list), len(abstract_list)))
  logging.info("Total time: {}, averaged time: {} seconds".format(total_time, total_time/len(result_json_list)))



run inference

In [ ]:
class Args:
  def __init__(self):
    self.num = 50
    self.start_index = None
    self.save_dir = "/content/drive/MyDrive/Colab Notebooks/W2R/extraction_finetune_unsloth/result_codestyle3"
    self.save_dir_rewrite = True
    self.repeat = 5

# Create an instance of Args
args = Args()
base_folder = args.save_dir

# Run the process_abstracts function
for i in range(args.repeat):
  print("Running experiment {}/{}".format(i+1, args.repeat))
  logging.info("Running experiment {}/{}".format(i+1, args.repeat))


  # create subfolder in the save_dir: run_{i}
  save_dir = os.path.join(base_folder, "run_{}".format(i))
  args.save_dir = check_make_dir(save_dir, exist_ok=args.save_dir_rewrite)

  # set logging file
  logger = logging.getLogger()
  if logger.hasHandlers():
    logger.handlers.clear()
  log_file = os.path.join(args.save_dir, "extraction_log.log")
  logging.basicConfig(filename=log_file, filemode='w', level=logging.INFO, format='%(message)s')

  # Log each argument and its value
  for arg, value in vars(args).items():
    logging.info(f'Argument {arg}: {value}')
    print(f'{arg}: {value}')
  logging.info("-------------"*5)

  process_abstracts(args)



Running experiment 1/5
num: 50
start_index: None
save_dir: /content/drive/MyDrive/Colab Notebooks/W2R/extraction_finetune_unsloth/result_codestyle3/run_0
save_dir_rewrite: True
repeat: 5
Processing abstract 0, time taken: 2.61 seconds
Processing abstract 1, time taken: 6.24 seconds
Processing abstract 2, time taken: 3.53 seconds
Processing abstract 3, time taken: 4.18 seconds
Processing abstract 4, time taken: 2.83 seconds
Processing abstract 5, time taken: 4.00 seconds
Processing abstract 6, time taken: 2.54 seconds
Processing abstract 7, time taken: 3.23 seconds
Processing abstract 8, time taken: 3.43 seconds
Processing abstract 9, time taken: 7.10 seconds
Processing abstract 10, time taken: 2.82 seconds
Processing abstract 11, time taken: 5.46 seconds
Processing abstract 12, time taken: 3.54 seconds
Processing abstract 13, time taken: 2.90 seconds
Processing abstract 14, time taken: 2.56 seconds
Processing abstract 15, time taken: 8.50 seconds
Processing abstract 16, time taken: 3.1